### Importing necessary libraries and modules
This code imports a variety of libraries, including PyTorch, NumPy, and Hugging Face's Transformers for model handling. It also contains commented-out code related to Curl and ONNX models.

In [ ]:
# import curl
import torch
import numpy as np
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BertTokenizer,
    pipeline,
    AutoModel,
)
# curl.init()
# from optimum.onnxruntime import ORTModelForSequenceClassification, ORTModel

### Defining and Loading a Pretrained Model
Here, we define `MODEL_NAME` with various BERT-based models for sequence classification. We load the model using Hugging Face's `AutoModelForSequenceClassification` class.

In [2]:
MODEL_NAME = "prajjwal1/bert-tiny"
MODEL_NAME = "gaunernst/bert-tiny-uncased"
#MODEL_NAME = "optimum/distilbert-base-uncased-finetuned-sst-2-english"
MODEL_NAME = "mrm8488/bert-tiny-finetuned-sms-spam-detection"
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

### Tokenizing Input Text
We prepare some input text and tokenize it using the model's tokenizer. The output tokens will be fed into the model for further processing. The `max_length` is set to the model's hidden size.

In [3]:
INPUT_TEXT = "Hello, my dog is cute"
hidden_size = model.config.hidden_size
inputs = tokenizer(
    INPUT_TEXT,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=hidden_size,
)

### Inspecting Tokenized Input
Here, we look at the tokenized representation of our input text.

In [ ]:
inputs["input_ids"]

### PyTorch to ONNX Conversion Function
This function converts a PyTorch model to ONNX format. It takes the PyTorch model, a tokenizer, and an optional max length for tokenization as arguments. We use a dummy input to simulate real input data during conversion.

In [ ]:
import tempfile, onnx

def convert_pytorch_to_onnx_with_tokenizer(model, tokenizer, max_length=hidden_size):
    """
    Converts a PyTorch model to ONNX format, using tokenizer output as input.
    """
    model.eval()
    dummy_input = "This is a sample input text for ONNX conversion."
    inputs = tokenizer(
        dummy_input,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )
    input_names = ["input_ids", "attention_mask"]
    onnx_file_path = tempfile.mktemp(suffix=".onnx")
    torch.onnx.export(
        model,
        tuple(inputs[k] for k in input_names),
        onnx_file_path,
        opset_version=20,
        do_constant_folding=True,
        input_names=input_names,
        output_names=["logits"],
        dynamic_axes={name: {0: "batch_size"} for name in input_names},
    )
    print(f"Model exported to {onnx_file_path}")
    return onnx_file_path, input_names

onnx_file_path, input_names = convert_pytorch_to_onnx_with_tokenizer(
    model, tokenizer, max_length=hidden_size
)

### Running the ONNX Model
Once we have converted the PyTorch model into ONNX format, we can run the ONNX model using the ONNX Runtime. This code prepares inputs for ONNX and asserts that the ONNX model produces outputs similar to the PyTorch model.

In [ ]:
expected_outputs = model(**inputs).logits
expected_outputs

### Function for Classifying Text Samples with ONNX
This function takes a sentence, tokenizes it, runs it through the ONNX model, and compares the ONNX model's outputs with the PyTorch model's expected outputs.

In [ ]:
import onnxruntime as ort
def classify_sample(sentence, model, onnx_file_path, input_names):
    inputs = tokenizer(
        sentence,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=hidden_size,
    )
    expected_outputs = model(**inputs).logits
    ort_session = ort.InferenceSession(onnx_file_path)
    inputs_onnx = {name: inputs[name].numpy() for name in input_names}
    outputs = ort_session.run(None, inputs_onnx)
    np.testing.assert_allclose(
        np.array(outputs[0]), expected_outputs.detach().numpy(), rtol=1e-03, atol=1e-05
    )
    return outputs
classify = lambda sentence: classify_sample(
    sentence, model, onnx_file_path, input_names
)
classify("My dog is so ugly")
classify("My food is bitter")
classify("The film I watched was very entertaining")
classify("")
classify("I am happy today")
classify("Get the latest news from your phone and earn now")
classify("Get the inheritance for the prince in Nigeria")
classify("You have won a lottery")
classify("You have won a lottery and need to pay a fee to claim it")
classify("You have won a lottery and need to pay a fee to claim it. Please provide your bank details")

### Trying the PyTorch Model
This section tries running a PyTorch model for text classification using Hugging Face's `pipeline` for comparison with ONNX.

In [8]:
# INPUT_TEXT = "this is a positive sentence about the movie"
# model.eval()
# torch_classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)
# torch_results = torch_classifier(INPUT_TEXT)
# print("TORCH:", torch_results)

### Trying the ONNX Model
We attempt to classify the same inputs using the ONNX model that was converted earlier from Hugging Face.

#### Trying PyTorch model from Huggingface
In this section, we try a text classification task using a pre-trained PyTorch model from Huggingface. We prepare the input text, pass it through the model, and retrieve the classification result.

In [8]:
# INPUT_TEXT = "this is a positive sentence about the movie"
# model.eval()
# torch_classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)
# torch_results = torch_classifier(INPUT_TEXT)
# print("TORCH:", torch_results)

# save_directory = "."
# onnx_model = ORTModelForSequenceClassification.from_pretrained(".", file_name="bert_tiny.onnx")
# onnx_classifier = pipeline('text-classification', model=onnx.load(onnx_file_path), tokenizer=tokenizer)
# onnx_results = onnx_classifier(INPUT_TEXT)
# print("ONNX :", onnx_results)
# tokens = tokenizer(
#     INPUT_TEXT,
#     return_tensors='pt',
# )
# NUM_INPUT_TOKENS = len(tokens)  # actual number of input tokens

# hidden_size = model.config.hidden_size
# # To ensure a constant input size, we will always pad or truncate to a fixed size
# encoded_input = tokenizer(
#     INPUT_TEXT,
#     return_tensors='pt',
#     padding="max_length",
#     truncation=True,
#     max_length=hidden_size,
# )

# # output = model.forward(
# #     input_ids=encoded_input["input_ids"],
# #     attention_mask=encoded_input["attention_mask"],
# # )

# encoded_input['input_ids'] = torch.ones_like(encoded_input['input_ids'])

# #print(encoded_input)
# output = model(**encoded_input)
# #print(output)
# predicted_classes = torch.argmax(output.logits)
# print("Predictions are: ", predicted_classes, torch.functional.softmax(output.logits))

#### Trying ONNX model converted from Huggingface
In this section, we try an ONNX version of the Huggingface model, perform the text classification task, and compare the results with the PyTorch version.

In [ ]:
import curl

curl.init()
print("ONNX Path: ", onnx_file_path)

In [ ]:
with open(onnx_file_path, "rb") as f:
    private_model = curl.nn.from_onnx(f)
    print(type(private_model))
    private_model.encrypt()
    print(type(private_model))


In [ ]:
onnx_model = onnx.load_model(onnx_file_path)
for i, node in enumerate(onnx_model.graph.node):
    print(f"[{i}] Operation: {node.op_type}")
    print(f"{node.name}")
    print(f"Inputs: {node.input}")
    print(f"Outputs: {node.output}")

In [ ]:
from curl.nn.onnx_executor import ONNXExecutor

onnx_executor = ONNXExecutor(onnx_model)
result = onnx_executor.forward(
    {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"]}
)

np.testing.assert_allclose(result, expected_outputs.detach().numpy(), rtol=1e-03, atol=1e-05)
print(result)

In [13]:
node_inputs = onnx_executor.get_node_inputs_by_name("/bert/encoder/layer.0/attention/self/Softmax_output_0")


In [ ]:

opset = set()
for i, node in enumerate(onnx_model.graph.node):
    # print(f"[>>>][{node.op_type}]")
    opset.add(node.op_type)
    # print(f"[>>>][{i}]")
    # print(f"Operation: {node.op_type} (node {i})")
    # print(f"Inputs: {node.input}")
    # print(f"Outputs: {node.output}")
    # print(f"[<<<][{i}]")
print(opset)

In [ ]:
private_model.eval()
private_input_ids = curl.cryptensor(inputs["input_ids"], precision=0)
private_attention_mask = curl.cryptensor(inputs["attention_mask"], precision=0)

private_input_ids.encoder._precision_bits

In [16]:
private_model.eval()
private_input_ids = curl.cryptensor(inputs["input_ids"], precision=0)
private_attention_mask = curl.cryptensor(inputs["attention_mask"], precision=0)
private_output = private_model(private_input_ids, private_attention_mask)

In [ ]:
expected_outputs = model(**inputs).logits
diff = expected_outputs - private_output.get_plain_text()
print("Diff: ", diff)
expected_outputs, private_output.get_plain_text()

In [ ]:
%%time
import onnxruntime as ort

l = []

def classify_sample(sentence, model, private_model):
    inputs = tokenizer(
        sentence,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=hidden_size,
    )
    expected_outputs = model(**inputs).logits

    private_model.eval()
    private_input_ids = curl.cryptensor(inputs["input_ids"], precision=0)
    private_attention_mask = curl.cryptensor(inputs["attention_mask"], precision=0)
    private_output = private_model(private_input_ids, private_attention_mask)
    private_output = private_output.get_plain_text().numpy()
    print(f"Expected outputs: {expected_outputs}")
    print(f"CRYPTEN model outputs: {private_output}")
    np.testing.assert_allclose(expected_outputs.detach().numpy(), private_output, rtol=1e-03, atol=1e-05)
    l.append(sentence)
    return expected_outputs, private_output

classify_sample("Sample sentence to classify", model, private_model)
